import all necessary package

In [63]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

use beautifulsoup to scrape the wiki page table

In [2]:
tor_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
rsp = requests.get(tor_url)

In [24]:
tor_soup = BeautifulSoup(rsp.content,'lxml')

In [27]:
table1 = tor_soup.find_all('table')[0]

In [29]:
data1 = pd.read_html(str(table1))

In [149]:
dataCode = data1[0]

dataCode is the data frame of the original data ,but need some clean up

In [150]:
dataCode.columns = dataCode.loc[0]

In [151]:
dataCode.head()

,Postcode,Borough,Neighbourhood
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [152]:
dataCode = dataCode.drop(0)

In [153]:
dataCode.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


remove all index with "not assigned" Borough

In [154]:
dataCode = dataCode.replace({'Borough':'Not assigned'},np.nan)

In [155]:
dataCode.head()

,Postcode,Borough,Neighbourhood
1,M1A,NaN,Not assigned
2,M2A,NaN,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [156]:
dataCode = dataCode.dropna(axis=0)

In [157]:
dataCode.head(10)

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


if the Neighbourhood is not assigned, change it to the Borough value in the same index

In [197]:
dataCode.loc[dataCode['Neighbourhood']=='Not assigned','Neighbourhood'] = dataCode[dataCode['Neighbourhood']=='Not assigned']['Borough']

In [198]:
dataCode[dataCode['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood


combine all neighbourhood the index with the same Borough and Postcode, use "," to seperate each

In [199]:
gp2 = dataCode.groupby('Postcode').agg({'Borough':'min','Neighbourhood':lambda x: ','.join(x)})

In [200]:
gp2.head(20)

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"


In [201]:
gp2.shape

(103, 2)

In [120]:
gp2.head()

,Borough,Neighbourhood
Postcode,,
M1B,ScarboroughScarborough,"Rouge,Malvern"
M1C,ScarboroughScarboroughScarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,ScarboroughScarboroughScarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


reset the index and give the final data2

In [202]:
data2 = pd.DataFrame(gp2)
data2.reset_index(inplace=True)
data2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [203]:
data2.shape

(103, 3)

In [206]:
import geocoder # import geocoder

In [232]:
gps_data = pd.read_csv('./Geospatial_data')
gps_data = gps_data.rename(columns={'Postal Code':'Postcode'})
gps_data.set_index('Postcode',inplace=True)
gps_data.head(4)

,Latitude,Longitude
Postcode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917


In [210]:
for eachrow in data2.index:
    postal_code = data2.loc[eachrow,'Postcode']

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    data2.loc[eachrow,'Lattitude']=latitude
    data2.loc[eachrow,'Longitude']=longitude



TypeError: 'NoneType' object is not subscriptable

In [225]:
data2.head(1)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"


In [224]:
data3 = data2.set_index('Postcode')

In [233]:
gps_data.head(5)

,Latitude,Longitude
Postcode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [226]:
data3.head(5)

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [234]:
data4= pd.concat([data3,gps_data],axis=1)

In [235]:
data4.head(10)

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476


In [238]:
data4 = data4.reset_index()